In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.copy()
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "ph2oimm",  # Maybe pH of the immersion liquid | pH (acidity or alkalinity)
        "Initial Setting Time",
        "flow",  # Maybe flow table test
        "residuo 24 micron",  # Maybe the residue left after passing the material through a sieve
        "R_wp",  # Maybe water to powder ratio
        "CS2",  # 2-day Compressive Strength
        "Soundness",
        # Removing One-Hot encoding variables
        "cem_type_CEM B",
        "cem_type_CEM C",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.953 (0.037)
MAE: -2.280 (0.027)
MAPE: -0.047 (0.001)
R2: 0.801 (0.005)


******
[TEST]
******
RMSE: -3.053 (0.145)
MAE: -2.373 (0.091)
MAPE: -0.049 (0.002)
R2: 0.785 (0.021)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation RMSE Test  \
                                                                     mean   
0  Chemical + Mineralogical  Linear Regression   Repeated KFold  3.053322   

             MAE Test           MAPE Test             R2 Test            
        std      mean       std      mean       std      mean       std  
0  0.144758  2.373273  0.091372  0.049463  0.002277  0.785187  0.020993

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.430 (0.259)
MAE: -1.844 (0.190)
MAPE: -0.038 (0.004)
R2: 0.860 (0.031)


******
[TEST]
******
RMSE: -3.877 (0.463)
MAE: -2.872 (0.595)
MAPE: -0.059 (0.012)
R2: 0.596 (0.116)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression              Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.876559  0.462525  2.872191  0.594589  0.059495  0.011507  0.596115   
1  3.053322  0.144758  2.373273  0.091372  0.049463  0.002277  0.785187   

             
        std  
0  0.115605  
1  0.020993

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.953 (0.082)
MAE: -2.268 (0.044)
MAPE: -0.047 (0.001)
R2: 0.796 (0.015)


******
[TEST]
******
RMSE: -3.577 (0.364)
MAE: -2.740 (0.280)
MAPE: -0.057 (0.007)
R2: 0.690 (0.075)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression              Repeated KFold   
2  Chemical + Mineralogical  Linear Regression           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.876559  0.462525  2.872191  0.594589  0.059495  0.011507  0.596115   
1  3.053322  0.144758  2.373273  0.091372  0.049463  0.002277  0.785187   
2  3.576885  0.363895  2.740066  0.280383  0.056762  0.006759  0.689686   

             
        std  
0  0.115605  
1  0.020993  
2  0.074950

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.877 (0.000)
MAE: 2.212 (0.000)
MAPE: 0.046 (0.000)
R2: 0.815 (0.000)


******
[TEST]
******
RMSE: 3.597 (0.000)
MAE: 2.944 (0.000)
MAPE: 0.062 (0.000)
R2: 0.669 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression           Out of time Split   
2  Chemical + Mineralogical  Linear Regression              Repeated KFold   
3  Chemical + Mineralogical  Linear Regression           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.876559  0.462525  2.872191  0.594589  0.059495  0.011507  0.596115   
1  3.597335  0.000000  2.943709  0.000000  0.061628  0.000000  0.669204   
2  3.053322  0.144758  2.373273  0.091372  0.049463  0.002277  0.785187   
3  3.576885  0.363895  2.740066  0.280383  0.056762  0.006759  0.689686   

             
        std  
0  0.115605  
1  0.000000  
2  0.020993  
3  0.074950

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 4.69233464e-01, -1.63321477e+00, -7.11977537e+00,  7.88301793e-02,
        3.05047727e+01, -3.56987591e+00,  2.89092889e-02,  7.45654321e+00,
       -2.75972414e+00, -4.89528173e+00,  8.32594737e+00,  5.55955327e+00,
        4.13516698e+00,  3.56438121e+00, -1.27258067e+00,  3.87839448e-01,
        1.48157937e+00,  8.55667756e-01,  8.02521568e-01,  5.09173981e-01,
        2.73050893e+00,  1.70937187e+00,  2.78909716e+00,  2.54462634e+01,
        1.07856998e-01, -2.19151644e-01])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

49.22584856396868

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Alite_M3 C3S M3', 'Alite_M1 C3S M1', 'Alite_Sum C3S tot',
       'Ratio_M1 (rapporto M1/M3)', 'C3S_CS (taglia dei cristalliti C3S)',
       'Belite_beta', 'C3A cub', 'C3A_ortho', 'C3A tot', 'C4AF', 'CaO',
       'Ca(OH)2', 'Calce libera', 'Periclasio (MgO)', 'Quartz', 'K2SO4',
       'Langbeinite – MgK2(SO4)2', 'Aphthitalite – (K,Na)3(SO4)2', 'Gesso',
       'Emiidrato', 'Anidrite', 'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(26,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(26,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Alite_Sum C3S tot,30.504773
Calcite – CaCO3,25.446263
C3A tot,8.325947
Belite_beta,7.456543
C4AF,5.559553
CaO,4.135167
Ca(OH)2,3.564381
Anidrite,2.789097
Gesso,2.730509
Emiidrato,1.709372


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression           Out of time Split   
2  Chemical + Mineralogical  Linear Regression              Repeated KFold   
3  Chemical + Mineralogical  Linear Regression           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.876559  0.462525   
1                                {"Test Size": 0.2}  3.597335  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  3.053322  0.144758   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.576885  0.363895   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.872191  0.594589  0.059495  0.011507  0.596115  0.115605  
1  2.943709  0.000000  0.061628  0.000000  0.669204  0.000000  
2  2.373273  0.091372  0.049463  0.002277  0.785187  0.020993  
3  2.740066  0.280383  0.056762  0.006759  0.689686  0.074950

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = "linear_regression_results_full_3.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = "linear_regression_results_grouped_3.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [27]:
df_results_to_save

Category Company Data Shape Timesteps                  Features  \
                                                                        
0  Local Model   partner_iv  (958, 26)       NaN  Chemical + Mineralogical   
1  Local Model   partner_iv  (958, 26)       NaN  Chemical + Mineralogical   
2  Local Model   partner_iv  (958, 26)       NaN  Chemical + Mineralogical   
3  Local Model   partner_iv  (958, 26)       NaN  Chemical + Mineralogical   

               Model            Cross Validation  \
                                                   
0  Linear Regression  Blocking Time Series Split   
1  Linear Regression           Out of time Split   
2  Linear Regression              Repeated KFold   
3  Linear Regression           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   2.430079  0.258676  ...   
1                                {"Test Size": 0.2}   2.877400  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.953194  0.036712  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.953140  0.081756  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.860188  0.031433  3.876559  0.462525  2.872191  0.594589  0.059495   
1  0.814954  0.000000  3.597335  0.000000  2.943709  0.000000  0.061628   
2  0.800553  0.005021  3.053322  0.144758  2.373273  0.091372  0.049463   
3  0.796296  0.014536  3.576885  0.363895  2.740066  0.280383  0.056762   

              R2 Test            
        std      mean       std  
0  0.011507  0.596115  0.115605  
1  0.000000  0.669204  0.000000  
2  0.002277  0.785187  0.020993  
3  0.006759  0.689686  0.074950  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>